In [1]:
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd
!wget https://chaabiv2.s3.ap-south-1.amazonaws.com/hiring/bigBasketProducts.csv
data = pd.read_csv("/content/bigBasketProducts.csv", usecols = ["category","sub_category","product","rating"])
data.head()

--2023-11-30 17:53:15--  https://chaabiv2.s3.ap-south-1.amazonaws.com/hiring/bigBasketProducts.csv
Resolving chaabiv2.s3.ap-south-1.amazonaws.com (chaabiv2.s3.ap-south-1.amazonaws.com)... 3.5.209.114, 16.12.40.50, 3.5.212.16, ...
Connecting to chaabiv2.s3.ap-south-1.amazonaws.com (chaabiv2.s3.ap-south-1.amazonaws.com)|3.5.209.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16739247 (16M) [text/csv]
Saving to: ‘bigBasketProducts.csv.1’

bigBasketProducts.c 100%[===================>]  15.96M  6.32MB/s    in 2.5s    

2023-11-30 17:53:18 (6.32 MB/s) - ‘bigBasketProducts.csv.1’ saved [16739247/16739247]



,product,category,sub_category,rating
0,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,4.1
1,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,2.3
2,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,3.4
3,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,3.7
4,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,4.4


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27555 entries, 0 to 27554
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product       27554 non-null  object 
 1   category      27555 non-null  object 
 2   sub_category  27555 non-null  object 
 3   rating        18929 non-null  float64
dtypes: float64(1), object(3)
memory usage: 861.2+ KB


In [4]:
data.nunique()

product         23540
category           11
sub_category       90
rating             40
dtype: int64

In [5]:
df = pd.DataFrame(data)
df['combined'] = df.apply(lambda row: ' '.join(map(str, [row['product'], row['category'], row['sub_category'], row['rating']])), axis=1)
df = df[['combined']]
df.to_csv('combined.csv', index=False)
df.head()

,combined
0,Garlic Oil - Vegetarian Capsule 500 mg Beauty ...
1,"Water Bottle - Orange Kitchen, Garden & Pets S..."
2,"Brass Angle Deep - Plain, No.2 Cleaning & Hous..."
3,Cereal Flip Lid Container/Storage Jar - Assort...
4,Creme Soft Soap - For Hands & Body Beauty & Hy...


In [6]:
!pip install -U transformers huggingface-hub hf_transfer

In [7]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Llama-2-7B-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf

/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf


In [8]:
!pip install langchain ctransformers sentence-transformers

In [9]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 200, 'repetition_penalty': 1.1, 'temperature':0.2, 'gpu_layers':0, 'context_length':3000}
llm = CTransformers(model='/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf'
      , model_type='llama2', config=config)

In [10]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter

loader = CSVLoader(file_path="/content/combined.csv")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=2100, chunk_overlap=100)
data_chunks = text_splitter.split_documents(data)

In [31]:
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

prompt_template = """
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{context}
{prompt}[/INST]

 """

prompt = PromptTemplate(
    input_variables=["context", "prompt"],
    template=prompt_template,
)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
!pip install ChromaDb
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(data_chunks, embeddings)

In [33]:
from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

In [ ]:
query = "Which soap are soft on hands?"
result = chain(query)
print(result['answer'])

In [17]:
!pip -q install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def chaabi():
  query = request.args.get('query', '')
  result = chain({"query": query})
  print(result['result'])
  return jsonify({'result': result['result']})
app.run()